# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
J. Neumann  ->  J. Neumann  |  ['J. Neumann']
J. Li  ->  J. Li  |  ['J. Li']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Li  ->  J. Li  |  ['J. Li']


Arxiv has 77 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.07395


extracting tarball to tmp_2510.07395...

 done.
Retrieving document from  https://arxiv.org/e-print/2510.07407


extracting tarball to tmp_2510.07407...

 done.


Found 99 bibliographic references in tmp_2510.07407/Le_Conte.bbl.
Issues with the citations
syntax error in line 662: '=' expected
Retrieving document from  https://arxiv.org/e-print/2510.07734


extracting tarball to tmp_2510.07734...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.07407-b31b1b.svg)](https://arxiv.org/abs/2510.07407) | **The evolution of the bar fraction and bar lengths in the last 12 billion years**  |
|| Z. A. L. Conte, et al. -- incl., <mark>J. Neumann</mark> |
|*Appeared on*| *2025-10-10*|
|*Comments*| *18 pages, 12 figures. Submitted to MNRAS*|
|**Abstract**|            We investigate the evolution of the bar fraction and length using an extended JWST NIRCam imaging dataset of galaxies in the $1 \leq z \leq 4$ redshift range. We assess the wavelength dependence of the bar fraction in disc galaxies and bar length evolution by selecting a nearly mass-complete CEERS disc sample and performing independent visual classifications on the short (F200W) and long (F356W+F444W) wavelength channels. A similar bar fraction is observed for both samples, and combined we find a declining trend in the bar fraction: $0.16^{+0.03}_{-0.03}$ at $1 \leq z < 2$; $0.08^{+0.02}_{-0.01}$ at $2 \leq z < 3$; $0.07^{+0.03}_{-0.01}$ at $3 \leq z \leq 4$. This corroborates our previous work and other recent studies, suggesting that dynamically cold and rotationally supported massive discs are present at Cosmic Noon. No evolution in the F356W+F444W bar length is measured from $z = 4$ to $z = 1$, which has a mean of 3.6\,kpc, but a slight increase of about 1\,kpc towards $z = 1$ is measured in the F200W sample, which has a mean of 2.9\,kpc. The bar sample is shorter in the short-wavelength channel due to the better physical spatial resolution; however, we also suggest that dust obscuration plays a role. We find that the correlation between bar length and galaxy mass for massive galaxies observed at $z < 1$ is not seen at $z > 1$. By adding samples of barred galaxies at $z<1$, we show that there is a modest increase in the bar length ($\approx 2$\,kpc) towards $z=0$, but bars longer than $\approx8$\,kpc are only found at $z<1$. We show that bars and discs grow in tandem, for the bar length normalised by disc size does not evolve from $z = 4$ to $z = 0$. Not only is a significant population of bars forming beyond $z = 1$, but our results also show that some of these bars are as long and strong as the average bar at $z\approx0$.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.07395-b31b1b.svg)](https://arxiv.org/abs/2510.07395) | **Unified Spectrospatial Forward Models: Spatially Continuous Maps of Weak Emission Lines in the Rosette Nebula with SDSS-V LVM**  |
|| T. Hilder, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2025-10-10*|
|*Comments*| *31 pages, 11 figures. Submitted to ApJ*|
|**Abstract**|            Analyses of IFU data are typically performed on a per-spaxel basis, with each spectrum modelled independently. For low signal-to-noise (S/N) features such as weak emission lines, estimating properties is difficult and imprecise. Arbitrary binning schemes boost S/N at the cost of resolution, and risk introducing biases. We present a general forward-modelling approach that assumes spectra close on the sky are more similar than distant ones, and so can be modelled jointly. These "spectrospatial" models exploit spatial correlation to provide robust inferences, while simultaneously providing continuous predictions of line properties like strength and kinematics across the sky. Instrumental and calibration systematics are straightforward to include and infer. The model provides a natural trade-off between spatial resolution and S/N in a data-driven way. We apply this to Sloan Digital Sky Survey V (SDSS-V) Local Volume Mapper (LVM) data of the Rosette Nebula, producing continuous maps of fluxes and kinematics for Balmer, nebular, and auroral lines, as well as weak C II and N II recombination lines, demonstrating the approach across three orders of magnitude in S/N, including in the very low-S/N regime. The method recovers identical morphologies across different lines tracing similar ionisation volumes, at varying resolutions set by the S/N. We additionally provide a general framework for building and fitting such models in JAX, suitable for many applications. The implementation is fast and memory efficient, scales to large data volumes as in LVM, and can be deployed on hardware accelerators.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.07734-b31b1b.svg)](https://arxiv.org/abs/2510.07734) | **A New Algol-type Binary with an Accretion disk**  |
|| T. He, et al. -- incl., <mark>J. Li</mark>, <mark>X. Zhang</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2025-10-10*|
|*Comments*| **|
|**Abstract**|            We present a comprehensive photometric and spectroscopic analysis of the Algol-type binary \textit{Gaia} DR3 1892576067672499328. We identified the system as a spectroscopic binary based on medium-resolution LAMOST spectra. Combined with \textit{TESS} photometry, we determine an orbital period of \( P = 2.47757 (1) \) days, a low mass ratio of \( q = 0.098 \pm 0.002 \), and an orbital inclination of \( i = 46.934^{+2.613}_{-1.11} \) degrees. The orbit is consistent with being circular (\( e = 0 \)). The binary comprises a \( M_1 = 1.817 ^{ +0.106}_{-0.202} \,M_\odot \), \( R_1 = 1.265^{+0.121}_{-0.160}\,R_\odot \) A-type primary and a Roche-lobe-filling secondary of \( M_2 = 0.179 ^{ +0.011}_{-0.020} \,M_\odot \), \( R_2 = 1.994 ^{ +0.041}_{-0.077} \,R_\odot \). The double-peak H$\alpha$ emission line indicates the possible existence of a Keplerian accretion disc. We established a simple standard accretion disc model and modeled the geometric and dynamical properties of the accretion disc. The obtained outer disc radius $R_{\mathrm{out}} \approx 3.36 \pm 0.43\,R_\odot$ is consistent with the values inferred from the emission velocity of H$\alpha$. Systemic velocity variations observed over time suggest the possible presence of a tertiary companion, with a minimum mass of $M_3 > 0.369 \pm 0.024 \,M_\odot$. Given the low mass ratio, the secondary may evolve into a proto-helium white dwarf, forming an \text{EL CVn}-type system in the future. This system offers valuable insights into accretion dynamics and the formation of binaries.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.07407/./Figures/R90_Lb_norm.jpg', 'tmp_2510.07407/./Figures/Length_mass.png', 'tmp_2510.07407/./Figures/bar_frac.jpg']
copying  tmp_2510.07407/./Figures/R90_Lb_norm.jpg to _build/html/
copying  tmp_2510.07407/./Figures/Length_mass.png to _build/html/
copying  tmp_2510.07407/./Figures/bar_frac.jpg to _build/html/
exported in  _build/html/2510.07407.md
    + _build/html/tmp_2510.07407/./Figures/R90_Lb_norm.jpg
    + _build/html/tmp_2510.07407/./Figures/Length_mass.png
    + _build/html/tmp_2510.07407/./Figures/bar_frac.jpg


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# The evolution of the bar fraction and bar lengths in the last 12 billion years

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.07407-b31b1b.svg)](https://arxiv.org/abs/2510.07407)<mark>Appeared on: 2025-10-10</mark> -  _18 pages, 12 figures. Submitted to MNRAS_

</div>
<div id="authors">

Z. A. L. Conte, et al. -- incl., <mark>J. Neumann</mark>

</div>
<div id="abstract">

**Abstract:** We investigate the evolution of the bar fraction and length using an extended JWST NIRCam imaging dataset of galaxies in the $1 \leq z \leq 4$ redshift range. We assess the wavelength dependence of the bar fraction in disc galaxies and bar length evolution by selecting a nearly mass-complete CEERS disc sample and performing independent visual classifications on the short (F200W) and long (F356W+F444W) wavelength channels. A similar bar fraction is observed for both samples, and combined we find a declining trend in the bar fraction: $0.16^{+0.03}_{-0.03}$ at $1 \leq z < 2$ ; $0.08^{+0.02}_{-0.01}$ at $2 \leq z < 3$ ; $0.07^{+0.03}_{-0.01}$ at $3 \leq z \leq 4$ . This corroborates our previous work and other recent studies, suggesting that dynamically cold and rotationally supported massive discs are present at Cosmic Noon. No evolution in the F356W+F444W bar length is measured from $z = 4$ to $z = 1$ , which has a mean of 3.6 kpc, but a slight increase of about 1 kpc towards $z = 1$ is measured in the F200W sample, which has a mean of 2.9 kpc. The bar sample is shorter in the short-wavelength channel due to the better physical spatial resolution; however, we also suggest that dust obscuration plays a role. We find that the correlation between bar length and galaxy mass for massive galaxies observed at $z < 1$ is not seen at $z > 1$ . By adding samples of barred galaxies at $z<1$ , we show that there is a modest increase in the bar length ( $\approx 2$ kpc) towards $z=0$ , but bars longer than $\approx8$ kpc are only found at $z<1$ . We show that bars and discs grow in tandem, for the bar length normalised by disc size does not evolve from $z = 4$ to $z = 0$ . Not only is a significant population of bars forming beyond $z = 1$ , but our results also show that some of these bars are as long and strong as the average bar at $z\approx0$ .

</div>

<div id="div_fig1">

<img src="tmp_2510.07407/./Figures/R90_Lb_norm.jpg" alt="Fig11" width="100%"/>

**Figure 11. -** The evolution of the bar length over $0 \leq z \leq 4$. The left panels show lengths measured in this work from JWST NIRCam filters F200W (blue) and F356W+F444W (red) for bars found in galaxies between the redshift range $1 \leq z \leq 4$, with their normalised distribution (see text for details) shown on the right panels. The first row shows the distribution of $R_{90}$, whilst the second row is the deprojected $L_{bar}$ and the third row is the normalised $L_{bar}$, $L_{bar}/R_{90}$. The high redshift sample is compared against a sample of SDSS $i-$band barred galaxies at $z \approx 0$\citepalias[][black]{Gadotti_2011} and a sample of barred galaxies at $0.2 < z \leq 0.835$ using F814W images from the COSMOS survey \citepalias[][grey]{Kim_2021}. The mean value for each parameter in each sample is given in the right panel, with the standard deviation. (*fig:length dist*)

</div>
<div id="div_fig2">

<img src="tmp_2510.07407/./Figures/Length_mass.png" alt="Fig7" width="100%"/>

**Figure 7. -** Bar length versus stellar mass for JWST NIRCam barred galaxies at $1 \leq z \leq 4$. The two JWST samples in F200W (crosses) and F356W+F444W (diamonds) with ordinary least squares fits (dash-dotted and dotted, respectively); binned mean L$_{bar}$ as boxes and standard deviation as error bars. The green dash-dot-dotted is the ordinary least squares fit to nearby SDSS barred galaxies from \citetalias{Gadotti_2011}. The grey solid line is the ordinary least squares bisector fit to barred galaxies between $0.2 \leq z \leq 0.8$ from \citetalias{Kim_2021}. The black dashed line shows the locally weighted regression fit for nearby S$^4$G  spiral galaxies from \citet{Erwin_2019}. The red and blue shaded regions show $2 \times FWHM$ for the lower and upper redshift boundaries of the F356W+F444W and F200W samples, respectively. The bottom panel shows the bar length stellar mass relation for local galaxies with bar ages estimated. (*fig: length-mass*)

</div>
<div id="div_fig3">

<img src="tmp_2510.07407/./Figures/bar_frac.jpg" alt="Fig10" width="100%"/>

**Figure 10. -** Evolution of the bar fraction in disc galaxies. At high-$z$ the bar fraction is found in three redshift bins, $1 \leq z < 2$, $2 \leq z < 3$ and $3 \leq z \leq 4$, using the visual classification of the images from the JWST NIRCam filters F356W+F444W (red stars) and F200W (blue stars) and combining these results to find a total bar fraction (yellow stars). Error bars in $f_{bar}$ are the $1\sigma$ bimodal interval, and the shaded area is the upper and lower bounds of the bar fraction (see text for details). Dashed horizontal error bars show the full range in $z$ of the identified bars, while thick horizontal solid lines show the corresponding 25\%-75\% inter-quartile range. The insert shows the bar fraction for the breakdown of strongly (right-pointing triangle) and weakly (left-pointing triangle) barred galaxies. The results of Paper 1 are black unfilled squares (HST) and stars (JWST), and in grey are the JWST bar fraction of \citetalias{Guo_2025}(diamond), \citetalias{Salcedo_2025}(circle) and \citetalias{Géron_2025}(square). (*fig:bar fraction*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.07407"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

124  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
